In [3]:
import sys
sys.path = ['/nfs/gns/homes/willj/anaconda3/envs/GTEx/lib/python3.5/site-packages'] + sys.path
# sys.path = ['/hps/nobackup/research/stegle/users/willj/GTEx/deps/qvalue/qvalue'] + sys.path
import pickle
import math
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy.stats import pearsonr
import os
%matplotlib inline
import qvalue

In [4]:
from statsmodels.sandbox.stats.multicomp import multipletests

In [5]:
GTEx_directory = '/hps/nobackup/research/stegle/users/willj/GTEx'

In [15]:
from pyensembl import EnsemblRelease

In [16]:
tissue = 'Lung'
if tissue == 'Artery - Tibial':
    tissue_filename = 'Artery_Tibial'
elif tissue == 'Heart - Left Ventricle':
    tissue_filename = 'Heart_Left_Ventricle'
elif tissue == 'Breast - Mammary Tissue':
    tissue_filename = 'Breast_Mammary_Tissue'
elif tissue == 'Brain - Cerebellum':
    tissue_filename = 'Brain_Cerebellum'
else:
    tissue_filename = tissue

tissue_expression_filepath = '/nfs/research2/stegle/stegle_secure/GTEx/download/49139/PhenoGenotypeFiles/RootStudyConsentSet_phs000424.GTEx.v6.p1.c1.GRU/ExpressionFiles/phe000006.v2.GTEx_RNAseq.expression-data-matrixfmt.c1/parse_data/44_tissues/GTEx_Data_20150112_RNAseq_RNASeQCv1.1.8_gene_rpkm_{}_normalised_without_inverse_gene_expression.txt'.format(tissue_filename)

with open(tissue_expression_filepath, 'r') as f:
    expression_table = np.array([x.split('\t') for x in f.read().splitlines()])
transcript_IDs = [x.split('.')[0] for x in expression_table[:,0][1:]]
data = EnsemblRelease(77)


gene_functions = []
for (i,ID) in enumerate(transcript_IDs):
    try:
        gene_functions.append(data.gene_by_id(ID).biotype)
    except ValueError:
        gene_functions.append('NA')
    if i % 1000 == 0:
        print (i)

protein_coding_IDs = np.array(transcript_IDs)[np.array(gene_functions) == 'protein_coding']
protein_coding_indexes = np.array([transcript_IDs.index(x) for x in protein_coding_IDs])

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000


In [58]:
tissue = 'Lung'
results = {}
results['large'] = {}
results['medium'] = {}
results['small'] = {}
for size in ['small','medium','large']:
    print (size)
    for aggregation in ['mean']:
        print (aggregation)
        all_real_pvalues = []
        for c in range(1024):
            if c%50 == 0:
                print (c)
            real_pvalues = np.array(pickle.load(open(GTEx_directory + '/data/processed/association_results/expression/inception_50_-1.h5/{}/{}/{}/pvalues/component{}_sh0_fl0.py'.format(size,tissue,aggregation,c),'rb'))[1])
            shuffled_pvalues = np.array(shuffled_data[1])

            all_real_pvalues.append(real_pvalues)
            results[size][aggregation] = {}
            results[size][aggregation]['real'] = real_pvalues
            results[size][aggregation]['shuffled'] = shuffled_pvalues

small
mean
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
medium
mean
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
large
mean
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000


In [59]:
min(results['large']['mean']['real'])

5.9445042314008272e-06

# Real 

## FDR 0.05

In [60]:

print (sum(multipletests(results['small']['mean']['real'], method='fdr_bh')[0]))
print (sum(multipletests(results['medium']['mean']['real'], method='fdr_bh')[0]))
print (sum(multipletests(results['large']['mean']['real'], method='fdr_bh')[0]))

0
19
0


## FDR 0.01

In [29]:

print (sum(multipletests(np.array(all_real_pvalues_small).flatten(), method='fdr_bh',alpha=0.01)[0]))
print (sum(multipletests(np.array(all_real_pvalues_medium).flatten(), method='fdr_bh',alpha=0.01)[0]))
print (sum(multipletests(np.array(all_real_pvalues_large).flatten(), method='fdr_bh',alpha=0.01)[0]))

203
4236
2363


# Shuffled

In [33]:
print (sum(multipletests(np.array(all_shuffled_pvalues_small).flatten(), method='fdr_bh')[0]))
print (sum(multipletests(np.array(all_shuffled_pvalues_medium).flatten(), method='fdr_bh')[0]))
print (sum(multipletests(np.array(all_shuffled_pvalues_large).flatten(), method='fdr_bh')[0]))

641
3757
8003


In [34]:
print (sum(multipletests(np.array(all_shuffled_pvalues_small).flatten(), method='fdr_bh', alpha=0.01)[0]))
print (sum(multipletests(np.array(all_shuffled_pvalues_medium).flatten(), method='fdr_bh', alpha=0.01)[0]))
print (sum(multipletests(np.array(all_shuffled_pvalues_large).flatten(), method='fdr_bh', alpha=0.01)[0]))

273
1990
2677
